# 第六章 生成回答

 - [一、基础介绍](#一、基础介绍)
     - [1.1 可视化理解](#1.1-可视化理解)
     - [1.2 文章数据](#1.2-文章数据)
 - [二、项目实战](#二、项目实战)
     - [2.1 环境设置](#2.1-环境设置)
     - [2.2 数据处理](#2.2-数据处理)
     - [2.3 搜索文章](#2.3-搜索文章)
     - [2.4 生成回复](#2.4-生成回复)


## 一、基础介绍

### 1.1 可视化理解

在本章课程中，我们将在检索系统的最后一个阶段引入 LLM 生成步骤。这样，我们就可以获得对问题的回答，而不仅仅是搜索结果。比如，在一些应用场景中，用户可以与文档、书籍，或者是在本课程中所涉及的文章进行对话。大型语言模型在很多方面都表现出色，但在某些情况下，它们可能需要一些额外的辅助。

In [3]:
question = "Are side projects important when you are starting to learn about AI?"
question_zh = "在你开始探索人工智能的学习之旅时，实战项目对你的学习成效有多大的促进作用？"

让我们来看一个例子：假设你有这样一个问题： "在你开始探索人工智能的学习之旅时，实战项目对你的学习成效有多大的促进作用？"

你完全可以将这个问题提交给大型语言模型。尽管这些模型可能会提供一些有趣的答案，但如果你能直接向专家提问或查阅专家的著作，那将更为有益。例如，你可以向吴恩达教授提问，或者查阅他关于这类问题的相关著作。

幸运的是，我们可以获取到吴恩达教授的一些著作。你可以在 DeepLearning AI 中找到一个名为 ["The Batch"](https://www.deeplearning.ai/the-batch/) 的新闻简报，以及一系列关于 ["How to Build a Career in AI.（如何规划 AI 领域的职业生涯）"](https://www.deeplearning.ai/the-batch/tag/letters/) 的文章。

![How to Build a Career in AI.](./images/6-1.png)

我们将运用本课程所学的知识，对这篇文章的内容进行搜索，并利用大型语言模型生成答案。

下面，我们通过可视化的方式来深入理解这个过程。


![Search can help LLMs in multiple ways](./images/6-2.png)

当我们向大型语言模型提出问题时，它能够生成多种不同的答案。

然而，有时我们可能希望它能够基于特定的文件或文档来生成答案。在这种情况下，你可以在生成答案之前添加一个搜索组件，以优化生成的结果。

![Where is the information stored?](./images/6-3.png)

当你完全依赖大型语言模型来获取答案时，你实际上是在利用它内部储存的全球信息。

![Search can add some context](./images/6-4.png)

接下来我们将看一下如何在代码中实现这一点。这是我们的问题。让我们建立我们的文本档案。对于这个用例，我们只需打开这些文章并复制文本。我们可以把所有的内容都复制进去，我们可以复制三份，因此这是第二篇文章。在这里我们有一个包含三篇文章文本的变量。你可以做更多。这个系列非常值得一读。可能有七八部分，但我们可以用三部分来进行这个示例。

然而，你可以通过预先提供上下文来增强模型的理解，例如在提示（prompt）中提供上下文。这种方法可以提高模型在特定领域、文章或文档中生成内容的质量，同时也能提高生成内容的事实性。

因此，在许多情况下，当你希望从模型中获取事实并将其与特定上下文关联时，这种方法可以提高模型生成内容的事实性。

这两个步骤之间的区别在于，我们首先将问题提交给一个搜索系统，就像我们在本课程中早期构建的那样。然后我们检索其中的一些结果，将它们连同问题一起放入 prompt 中，然后让生成模型根据这些上下文生成相应的回答。

### 1.2 文章数据

接下来我们将动手实现这部分内容。我们用吴恩达老师的前 3 篇文章作为检索文章。当然我们非常推荐阅读整个系列。


In [4]:
text = """
The rapid rise of AI has led to a rapid rise in AI jobs, and many people are building exciting careers in this field. A career is a decades-long journey, and the path is not always straightforward. Over many years, I’ve been privileged to see thousands of students as well as engineers in companies large and small navigate careers in AI. In this and the next few letters, I’d like to share a few thoughts that might be useful in charting your own course.

Three key steps of career growth are learning (to gain technical and other skills), working on projects (to deepen skills, build a portfolio, and create impact) and searching for a job. These steps stack on top of each other:

Initially, you focus on gaining foundational technical skills.
After having gained foundational skills, you lean into project work. During this period, you’ll probably keep learning.
Later, you might occasionally carry out a job search. Throughout this process, you’ll probably continue to learn and work on meaningful projects.
These phases apply in a wide range of professions, but AI involves unique elements. For example:

AI is nascent, and many technologies are still evolving. While the foundations of machine learning and deep learning are maturing — and coursework is an efficient way to master them — beyond these foundations, keeping up-to-date with changing technology is more important in AI than fields that are more mature.
Project work often means working with stakeholders who lack expertise in AI. This can make it challenging to find a suitable project, estimate the project’s timeline and return on investment, and set expectations. In addition, the highly iterative nature of AI projects leads to special challenges in project management: How can you come up with a plan for building a system when you don’t know in advance how long it will take to achieve the target accuracy? Even after the system has hit the target, further iteration may be necessary to address post-deployment drift.
While searching for a job in AI can be similar to searching for a job in other sectors, there are some differences. Many companies are still trying to figure out which AI skills they need and how to hire people who have them. Things you’ve worked on may be significantly different than anything your interviewer has seen, and you’re more likely to have to educate potential employers about some elements of your work.
Throughout these steps, a supportive community is a big help. Having a group of friends and allies who can help you — and whom you strive to help — makes the path easier. This is true whether you’re taking your first steps or you’ve been on the journey for years.

I’m excited to work with all of you to grow the global AI community, and that includes helping everyone in our community develop their careers. I’ll dive more deeply into these topics in the next few weeks.

Last week, I wrote about key steps for building a career in AI: learning technical skills, doing project work, and searching for a job, all of which is supported by being part of a community. In this letter, I’d like to dive more deeply into the first step.

More papers have been published on AI than any person can read in a lifetime. So, in your efforts to learn, it’s critical to prioritize topic selection. I believe the most important topics for a technical career in machine learning are:

Foundational machine learning skills. For example, it’s important to understand models such as linear regression, logistic regression, neural networks, decision trees, clustering, and anomaly detection. Beyond specific models, it’s even more important to understand the core concepts behind how and why machine learning works, such as bias/variance, cost functions, regularization, optimization algorithms, and error analysis.
Deep learning. This has become such a large fraction of machine learning that it’s hard to excel in the field without some understanding of it! It’s valuable to know the basics of neural networks, practical skills for making them work (such as hyperparameter tuning), convolutional networks, sequence models, and transformers.
Math relevant to machine learning. Key areas include linear algebra (vectors, matrices, and various manipulations of them) as well as probability and statistics (including discrete and continuous probability, standard probability distributions, basic rules such as independence and Bayes rule, and hypothesis testing). In addition, exploratory data analysis (EDA) — using visualizations and other methods to systematically explore a dataset — is an underrated skill. I’ve found EDA particularly useful in data-centric AI development, where analyzing errors and gaining insights can really help drive progress! Finally, a basic intuitive understanding of calculus will also help. In a previous letter, I described how the math needed to do machine learning well has been changing. For instance, although some tasks require calculus, improved automatic differentiation software makes it possible to invent and implement new neural network architectures without doing any calculus. This was almost impossible a decade ago.
Software development. While you can get a job and make huge contributions with only machine learning modeling skills, your job opportunities will increase if you can also write good software to implement complex AI systems. These skills include programming fundamentals, data structures (especially those that relate to machine learning, such as data frames), algorithms (including those related to databases and data manipulation), software design, familiarity with Python, and familiarity with key libraries such as TensorFlow or PyTorch, and scikit-learn.
This is a lot to learn! Even after you master everything in this list, I hope you’ll keep learning and continue to deepen your technical knowledge. I’ve known many machine learning engineers who benefitted from deeper skills in an application area such as natural language processing or computer vision, or in a technology area such as probabilistic graphical models or building scalable software systems.

How do you gain these skills? There’s a lot of good content on the internet, and in theory reading dozens of web pages could work. But when the goal is deep understanding, reading disjointed web pages is inefficient because they tend to repeat each other, use inconsistent terminology (which slows you down), vary in quality, and leave gaps. That’s why a good course — in which a body of material has been organized into a coherent and logical form — is often the most time-efficient way to master a meaningful body of knowledge. When you’ve absorbed the knowledge available in courses, you can switch over to research papers and other resources.

Finally, keep in mind that no one can cram everything they need to know over a weekend or even a month. Everyone I know who’s great at machine learning is a lifelong learner. In fact, given how quickly our field is changing, there’s little choice but to keep learning if you want to keep up. How can you maintain a steady pace of learning for years? I’ve written about the value of habits. If you cultivate the habit of learning a little bit every week, you can make significant progress with what feels like less effort.

In the last two letters, I wrote about developing a career in AI and shared tips for gaining technical skills. This time, I’d like to discuss an important step in building a career: project work.

It goes without saying that we should only work on projects that are responsible and ethical, and that benefit people. But those limits leave a large variety to choose from. I wrote previously about how to identify and scope AI projects. This and next week’s letter have a different emphasis: picking and executing projects with an eye toward career development.

A fruitful career will include many projects, hopefully growing in scope, complexity, and impact over time. Thus, it is fine to start small. Use early projects to learn and gradually step up to bigger projects as your skills grow.

When you’re starting out, don’t expect others to hand great ideas or resources to you on a platter. Many people start by working on small projects in their spare time. With initial successes — even small ones — under your belt, your growing skills increase your ability to come up with better ideas, and it becomes easier to persuade others to help you step up to bigger projects.

What if you don’t have any project ideas? Here are a few ways to generate them:

Join existing projects. If you find someone else with an idea, ask to join their project.
Keep reading and talking to people. I come up with new ideas whenever I spend a lot of time reading, taking courses, or talking with domain experts. I’m confident that you will, too.
Focus on an application area. Many researchers are trying to advance basic AI technology — say, by inventing the next generation of transformers or further scaling up language models — so, while this is an exciting direction, it is hard. But the variety of applications to which machine learning has not yet been applied is vast! I’m fortunate to have been able to apply neural networks to everything from autonomous helicopter flight to online advertising, partly because I jumped in when relatively few people were working on those applications. If your company or school cares about a particular application, explore the possibilities for machine learning. That can give you a first look at a potentially creative application — one where you can do unique work — that no one else has done yet.
Develop a side hustle. Even if you have a full-time job, a fun project that may or may not develop into something bigger can stir the creative juices and strengthen bonds with collaborators. When I was a full-time professor, working on online education wasn’t part of my “job” (which was doing research and teaching classes). It was a fun hobby that I often worked on out of passion for education. My early experiences recording videos at home helped me later in working on online education in a more substantive way. Silicon Valley abounds with stories of startups that started as side projects. So long as it doesn’t create a conflict with your employer, these projects can be a stepping stone to something significant.
Given a few project ideas, which one should you jump into? Here’s a quick checklist of factors to consider:

Will the project help you grow technically? Ideally, it should be challenging enough to stretch your skills but not so hard that you have little chance of success. This will put you on a path toward mastering ever-greater technical complexity.
Do you have good teammates to work with? If not, are there people you can discuss things with? We learn a lot from the people around us, and good collaborators will have a huge impact on your growth.
Can it be a stepping stone? If the project is successful, will its technical complexity and/or business impact make it a meaningful stepping stone to larger projects? (If the project is bigger than those you’ve worked on before, there’s a good chance it could be such a stepping stone.)
Finally, avoid analysis paralysis. It doesn’t make sense to spend a month deciding whether to work on a project that would take a week to complete. You'll work on multiple projects over the course of your career, so you’ll have ample opportunity to refine your thinking on what’s worthwhile. Given the huge number of possible AI projects, rather than the conventional “ready, aim, fire” approach, you can accelerate your progress with “ready, fire, aim.”

"""

中文版来自[吴恩达老师的知乎文章](https://www.zhihu.com/people/wu-en-da-89/posts)
1. [吴恩达来信：制定AI职业规划的小 tips](https://zhuanlan.zhihu.com/p/535874295)
2. [吴恩达来信: 做个终身学习者吧！](https://zhuanlan.zhihu.com/p/539093501)
3. [吴恩达来信：发展 AI 职业生涯的重要步骤——项目实践](https://zhuanlan.zhihu.com/p/541565826)

In [5]:
text_zh = """
人工智能的迅速崛起导致了相关工作的迅速增加，许多人开始在这个领域建立起令人兴奋的职业生涯。职业生涯是一个长达几十年的旅程，且道路并不总是一帆风顺的。多年来，我有幸看到成千上万的学生以及来自不同规模公司的工程师在人工智能领域工作。在今天这封信和接下来的几封信中，我将与大家分享一些可能对你制定职业规划有用的想法。

职业发展的三个关键步骤是学习（获得技术和其他技能），参与项目（深化技能，建立一个组合，创造影响）和找到一份合适的工作。将这些步骤叠加在一起：

最初，你需要专注于获得基本的技术技能。
在获得基本技能之后，你开始投入项目工作。在此期间，你可能会继续学习。
之后，你可能偶尔会进行求职。在这个过程中，你可能会继续学习并在有意义的项目上工作。
这些阶段适用于广泛的职业，但人工智能涉及独特的元素。例如:

人工智能刚刚起步，许多技术仍在发展。虽然机器学习和深度学习的基础正在成熟（上课是掌握它们的有效途径），但在这些基础之外，与更成熟的领域相比，与不断变化的技术保持同步在人工智能领域更为重要。
项目实践通常意味着与缺乏 AI 专业知识的利益相关者合作。这使得我们很难找到一个合适的项目、准确估计项目的时间和投资回报以及设定预期。此外，AI 项目的高度迭代特性导致了项目管理中的特殊挑战：当你无法提前知道需要多长时间才能达到目标精度时，如何才能提出一个构建系统的计划？即使系统已经达到目标，进一步的迭代可能是必要的，目的是应对部署后可能发生的漂移问题。
虽然在人工智能领域找工作与在其他领域找工作类似，但也有一些区别。许多公司仍在努力弄清楚他们需要哪些人工智能技能，以及如何雇佣拥有这些技能的人。你实践过的事情可能和面试官看到的有很大的不同，你可能更需要向潜在雇主介绍你工作中的一些细节。

纵观这些步骤，一个可以相互支持的社区也是一个很大的帮助。拥有一群可以帮助你的朋友和盟友（当然你也努力去帮助他们）会让这条路更好走。无论你是刚刚迈出人生的第一步，还是已经在这条路上走了好几年，一个可以共同进步的社区对你来说都是大有裨益的。

我很高兴能与你们所有人一起发展全球人工智能社区，包括帮助我们社区的每个人发展他们的职业生涯。我将在接下来的几周更深入地探讨这些话题。

在上周的来信中，我讨论了关于在人工智能领域建立职业生涯的关键步骤：学习技术技能、开展项目实践和寻找工作，所有这些都是通过成为 AI 社区的一员为支撑的。在本周的来信中，我想更深入地探讨第一步——学习技术技能。

在人工智能领域发表的论文比任何人一生能读到的都要多。因此，在你努力学习的过程中，选择一个专注的方向是至关重要的。我认为机器学习技术行业中最重要的主题是：

基础机器学习技术。例如，了解线性回归、逻辑回归、神经网络、决策树、聚类和异常检测等模型很重要。除了特定模型之外，更重要的是了解机器学习如何及为什么工作的核心概念，例如偏差/方差、损失函数、正则化、优化算法和错误分析。
深度学习。这已经占据了机器学习的很大一部分，如果不了解它，就很难在该领域出类拔萃！了解神经网络的基础知识、使其运作的实用技能（如超参数调整）、卷积网络、序列模型和 transformer 都是非常重要的。
与机器学习相关的数学知识。关键内容包括线性代数（向量、矩阵和它们的各种使用）以及概率和统计学（包括离散和连续概率、标准概率分布、基本规则，如独立性和贝叶斯定理，以及假设测试）。此外，探索性数据分析（exploratory data analysis, EDA，使用可视化和其他方法系统探索数据集）是一种被低估的技术。我发现 EDA 在以数据为中心的人工智能开发中特别实用，在这种开发中，分析错误和获得见解可以真正帮助推动进步！最后，对微积分的基本直观理解也会对你有所帮助。在之前的一封信中，我描述了学好机器学习所需的数学知识是如何随之变化的。例如，尽管有些任务需要微积分，但改进后的自动微分软件使发明和实现新的神经网络架构成为可能，且无需进行任何微积分操作。这在十年前几乎是不可能的。
软件开发。虽然你只需要机器学习建模技能就可以找到工作并做出巨大贡献，但如果你同时能编写好的软件来实现复杂的人工智能系统，那么你的工作机会就会进一步增加。这些技能包括编程基础知识、数据结构（尤其是与机器学习相关的数据结构，如数据帧）、算法（包括与数据库和数据操作相关的算法）、软件设计、熟悉 Python 以及熟悉关键库（如 TensorFlow 或 PyTorch）和 scikit-learn。

我们有很多东西要学！即使你掌握了这份清单中的所有内容，我也希望你能继续学习，继续加深你的技术知识。我认识许多机器学习工程师，他们在很多领域，例如应用领域（如自然语言处理或计算机视觉）或技术领域（如概率图形模型或构建可扩展的软件系统）都 A 得了更高级的技能。

如何获得这些技能？互联网上有很多好的内容，理论上阅读几十个网页都是很有帮助的。但是，当你的目标是深入理解时，阅读不连贯的网页其实是低效的，因为它们往往会相互重复、使用不一致的术语（这会减慢你的学习速度）、内容质量参差不齐等。这就是为什么一门好的课程——在这门课程中，一系列学习材料被组织成一种连贯和逻辑的形式——往往是掌握一个有意义的知识体系最有效的方式。当你吸收了课程中的可用知识后，你可以转向研究论文和其他资源。

最后，请记住，没有人能在一个周末甚至一个月内把他们需要知道的一切塞满大脑。我认识的每一个擅长机器学习的人都是终身学习者。事实上，考虑到我们的领域变化如此之快，如果你想跟上，除了继续学习之外别无他法。如何保持多年稳定的学习节奏？我写过关于“习惯的价值”的文章（The Best Way to Build a New Habit）。如果你养成了每周学习一会的习惯，就能用更少的努力获取显著的进步。

在最近的两封信中，我谈及了有关在人工智能领域发展职业生涯的内容，并分享了获得技术技能的技巧。今天，我想着重讨论一下职业生涯中的一个重要步骤：项目实践。

不言而喻的是，我们应当只从事负责任、合乎道德、有益于大众的项目。但在这些限制下我们依然有很多选择。此前我提到过如何识别和界定人工智能项目。本周的来信和下周的来信会有不同的重点：挑选和执行项目时要着眼于职业发展。

一个富有成果的职业生涯将包括许多项目，我们希望随着时间的推移这些项目能在范围、复杂性和影响力方面不断扩大。因此，从小处着手是较为有利的。利用早期项目进行学习，随着技能的增长，逐步迈向更大的项目。

当你开始创业时，不要寄希望于别人会把好的想法或资源放在盘子里递给你。许多人都是从利用业余时间做一些小项目开始的。随着最初的成功，即便是微小的成功，不断提高的技能也提升了你提出更好想法的能力，并且更容易说服他人帮助你攻克更大的项目。

如果你没有任何项目想法呢？可以考虑从以下几点入手：

加入现有项目。如果你遇到其他有想法的人，请询问是否能加入他们的项目。
保持阅读和与人交谈。每当我花大量时间阅读、学习课程或与领域专家交谈时，我就会迸发出新的灵感。相信你也会的。
关注应用领域。许多研究人员正在努力推进基本的人工智能技术——比如，通过发明下一代 transformers 或进一步扩大语言模型——虽然这是一个令人兴奋的方向，但很切实难做到。但是，机器学习尚未涉足的各种应用是潜力巨大的！幸运的是，我能够将神经网络应用于从无人驾驶直升机到在线广告的所有领域，部分原因是我是在很少有人开发这些应用程序的时候加入的。如果你所在的公司或学校关心某个特定的应用程序，请有针对性地探索机器学习的可能性。这可以让你第一眼看到一个潜在的、其他人还没有做过的创造性应用——一个你可以进行独特操作的应用。
开发副业。即使你拥有一份全职工作，利用业余时间从事一个有趣的项目，不管这个项目最终能不能有大发展，都可以帮你激发创造力，并加强和其他人工智能开发者的合作。当我还是一名全职教授时，从事在线教育并不是我“工作”的一部分，从事研究和授课理论上才是我的本职工作。但出于对教育的热情，这成了我的一个有趣的爱好。早期在家录制视频的经历帮助我后来以更实质性的方式从事在线教育。硅谷有很多创业公司的故事，这些创业公司都是依托辅助项目起步的。只要不与你的雇主产生冲突，这些项目就可以成为重要事情的垫脚石。

如果你有一些项目想法，应该选择哪一个呢？以下是你需要考虑的因素的快速清单：

这个项目会帮助你在技术上成长吗？理想情况下，它应该具有足够的挑战性来拓展你的技能，但也不能太困难，以至于你几乎没有成功的机会。这将使你走上掌握更大技术复杂性的道路。
你有好的队友一起工作吗？如果没有，有人可以和你进行讨论吗？我们从周围的人身上学到了很多，优秀的合作者将对你的成长产生巨大影响。
它能成为垫脚石吗？如果项目获得成功，其技术复杂性和/或业务影响是否会使其成为大型项目的重要垫脚石？（如果这个项目比你以前做过的要大，那么它很有可能成为这样一个垫脚石。）

最后，避免分析瘫痪。耗费一个月的时间来决定是否要完成一个只需一周时间就能完成的项目是没有意义的。在你的职业生涯中，你将从事多个项目，因此你将有足够的机会完善你的思想，了解哪些是值得的。考虑到大量可能的人工智能项目，与其使用传统的“准备、瞄准、发射”方法，你更可以通过“准备、发射、瞄准”的步骤来加快进度。

"""

## 二、项目实战

### 2.1 环境设置

安装必须的 Python 库并且准备好后续内容所需要的 API-KEY

* Cohere 获取 embedding
* dotenv 库是用来检查环境变量的~
* numpy 对数据进行处理
* warnings 控制警告信息

In [6]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # 读取本地的 .env 文件

In [7]:
import cohere

import numpy as np
import warnings
warnings.filterwarnings('ignore') # 忽略不必要的警告信息

### 2.2 数据处理

跟之前课程学到的一样，我们通过下面的步骤对数据完成处理
1. 分块
2. 生成 embedding
3. 构建语义索引



#### 2.2.1 分块 Chunking

In [8]:
# 将文本拆分成段落列表
texts = text.split('\n\n')

# 移除空格和换行符
texts = np.array([t.strip(' \n') for t in texts if t])

现在，让我们看一下前三个分块。

In [9]:
print(texts[:3])

['The rapid rise of AI has led to a rapid rise in AI jobs, and many people are building exciting careers in this field. A career is a decades-long journey, and the path is not always straightforward. Over many years, I’ve been privileged to see thousands of students as well as engineers in companies large and small navigate careers in AI. In this and the next few letters, I’d like to share a few thoughts that might be useful in charting your own course.'
 'Three key steps of career growth are learning (to gain technical and other skills), working on projects (to deepen skills, build a portfolio, and create impact) and searching for a job. These steps stack on top of each other:'
 'Initially, you focus on gaining foundational technical skills.\nAfter having gained foundational skills, you lean into project work. During this period, you’ll probably keep learning.\nLater, you might occasionally carry out a job search. Throughout this process, you’ll probably continue to learn and work on 

In [10]:
# 将文本拆分成段落列表
texts_zh = text_zh.split('\n\n')

# 移除空格和换行符
texts_zh = np.array([t.strip(' \n') for t in texts_zh if t])
print(texts_zh[:3])

['人工智能的迅速崛起导致了相关工作的迅速增加，许多人开始在这个领域建立起令人兴奋的职业生涯。职业生涯是一个长达几十年的旅程，且道路并不总是一帆风顺的。多年来，我有幸看到成千上万的学生以及来自不同规模公司的工程师在人工智能领域工作。在今天这封信和接下来的几封信中，我将与大家分享一些可能对你制定职业规划有用的想法。'
 '职业发展的三个关键步骤是学习（获得技术和其他技能），参与项目（深化技能，建立一个组合，创造影响）和找到一份合适的工作。将这些步骤叠加在一起：'
 '最初，你需要专注于获得基本的技术技能。\n在获得基本技能之后，你开始投入项目工作。在此期间，你可能会继续学习。\n之后，你可能偶尔会进行求职。在这个过程中，你可能会继续学习并在有意义的项目上工作。\n这些阶段适用于广泛的职业，但人工智能涉及独特的元素。例如:']


#### 2.2.2 生成 embedding

接下来，我们用 Cohere 生成文本的 embedding

In [11]:
co = cohere.Client(os.environ["COHERE_API_KEY"])

# 获取 embedding
response = co.embed(
    texts=texts.tolist(),
).embeddings

default model on embed will be deprecated in the future, please specify a model in the request.


In [58]:
response_zh = co.embed(
    texts=texts_zh.tolist(),
    model="embed-multilingual-v2.0"
).embeddings

#### 2.2.3构建检索索引

导入对应 Python 库。
Annoy 向量搜索库。
NumPy，Pandas 用来做数据处理。

In [12]:
from annoy import AnnoyIndex
import numpy as np
import pandas as pd

这里把 embedding 变成了一个 NumPy 数组。然后建了一个新的向量索引。并将向量插入其中，最后构建它并保存到文件中。

In [13]:
# 将返回的响应转成 array 的数据格式，并且同时对 embedding 的维度进行校验，确保一致
embeds = np.array(response)

# 根据 embedding 的大小创建索引
search_index = AnnoyIndex(embeds.shape[1], 'angular')
# 将 embeddings 添加到搜索索引中
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees
search_index.save('test.ann')

True

In [50]:
# 将返回的响应转成 array 的数据格式，并且同时对 embedding 的维度进行校验，确保一致
embeds_zh = np.array(response_zh)

# 根据 embedding 的大小创建索引
search_index_zh = AnnoyIndex(embeds_zh.shape[1], 'angular')
# 将 embeddings 添加到搜索索引中
for i in range(len(embeds_zh)):
    search_index_zh.add_item(i, embeds_zh[i])

search_index_zh.build(10) # 10 trees
search_index_zh.save('test_zh.ann')

True

### 2.3 搜索文章

接下来让我们定义一个函数来对吴恩达老师的文章进行检索。

我们用一个 query 在这个数据集上进行搜索。为了做到这一点，这些步骤正是我们过去所见到的：
1. 对 query 生成 embedding
2. 在文档索引中进行向量检索
3. 比较 query 与文本中每个段落的 embedding 相似度，
4. 返回按照相似度排序的结果。

In [42]:
def search_andrews_article(query):
    """
    根据给定的 query，检索与吴恩达老师文章中与之最相似的内容。

    参数:
    query (str): 用户的查询字符串。

    返回:
    search_results (list): 与查询最相似的文章列表。

    """

    # 获取 query 的 embeddings
    query_embed = co.embed(texts=[query]).embeddings

    # 检索最近的 embedding 结果
    similar_item_ids = search_index.get_nns_by_vector(query_embed[0],
                                                    10,
                                                  include_distances=True)

    # 从文本中获取搜索结果
    search_results = texts[similar_item_ids[0]]

    return search_results

In [59]:
def search_andrews_article_zh(query):
    """
    根据给定的 query，检索与吴恩达老师文章中与之最相似的内容。

    参数:
    query (str): 用户的查询字符串。

    返回:
    search_results (list): 与查询最相似的文章列表。

    """

    # 获取 query 的 embeddings
    query_embed = co.embed(texts=[query], model='embed-multilingual-v2.0').embeddings

    # 检索最近的 embedding 结果（使用不同的索引）
    similar_item_ids = search_index_zh.get_nns_by_vector(query_embed[0],
                                                    10,
                                                  include_distances=True)

    # 从文本中获取搜索结果
    search_results = texts_zh[similar_item_ids[0]]

    return search_results

现在我们可以向这个搜索系统提问，我们都很想知道吴恩达老师对此会有什么看法。

在这里我们返回了第一个结果。这是一个很长的段落，与我们的问题匹配度最高。这里提到了即使在全职工作的情况下，也可以开发副业项目。这样的项目，无论最终是否能发展成更大的事物，都能激发我们的创造力。

In [44]:
results = search_andrews_article(
    "Are side projects a good idea when trying to build a career in AI?"
)

print(results[0])

default model on embed will be deprecated in the future, please specify a model in the request.


Join existing projects. If you find someone else with an idea, ask to join their project.
Keep reading and talking to people. I come up with new ideas whenever I spend a lot of time reading, taking courses, or talking with domain experts. I’m confident that you will, too.
Focus on an application area. Many researchers are trying to advance basic AI technology — say, by inventing the next generation of transformers or further scaling up language models — so, while this is an exciting direction, it is hard. But the variety of applications to which machine learning has not yet been applied is vast! I’m fortunate to have been able to apply neural networks to everything from autonomous helicopter flight to online advertising, partly because I jumped in when relatively few people were working on those applications. If your company or school cares about a particular application, explore the possibilities for machine learning. That can give you a first look at a potentially creative applicatio

In [81]:
results = search_andrews_article_zh(
    "在你开始探索人工智能的学习之旅时，实战项目对你的学习成效有多大的促进作用？"
)

print(results[0])

在最近的两封信中，我谈及了有关在人工智能领域发展职业生涯的内容，并分享了获得技术技能的技巧。今天，我想着重讨论一下职业生涯中的一个重要步骤：项目实践。


下面是英文结果对应的中文片段

In [92]:
context_zh = """
加入现有项目。如果你遇到其他有想法的人，请询问是否能加入他们的项目。
保持阅读和与人交谈。每当我花大量时间阅读、学习课程或与领域专家交谈时，我就会迸发出新的灵感。相信你也会的。
关注应用领域。许多研究人员正在努力推进基本的人工智能技术——比如，通过发明下一代 transformers 或进一步扩大语言模型——虽然这是一个令人兴奋的方向，但很切实难做到。但是，机器学习尚未涉足的各种应用是潜力巨大的！幸运的是，我能够将神经网络应用于从无人驾驶直升机到在线广告的所有领域，部分原因是我是在很少有人开发这些应用程序的时候加入的。如果你所在的公司或学校关心某个特定的应用程序，请有针对性地探索机器学习的可能性。这可以让你第一眼看到一个潜在的、其他人还没有做过的创造性应用——一个你可以进行独特操作的应用。
开发副业。即使你拥有一份全职工作，利用业余时间从事一个有趣的项目，不管这个项目最终能不能有大发展，都可以帮你激发创造力，并加强和其他人工智能开发者的合作。当我还是一名全职教授时，从事在线教育并不是我“工作”的一部分，从事研究和授课理论上才是我的本职工作。但出于对教育的热情，这成了我的一个有趣的爱好。早期在家录制视频的经历帮助我后来以更实质性的方式从事在线教育。硅谷有很多创业公司的故事，这些创业公司都是依托辅助项目起步的。只要不与你的雇主产生冲突，这些项目就可以成为重要事情的垫脚石。

如果你有一些项目想法，应该选择哪一个呢？以下是你需要考虑的因素的快速清单：
"""

这就是这篇长篇文章的核心答案。这是一个展示如何利用大型语言模型回答问题的优秀示例。我们可以将问题提交给模型，让它为我们提取出相关的信息。下面，让我们来实践一下。



### 2.4 生成回复

我们将创建一个新函数，用于向吴恩达老师提出问题。在此函数中，我们会设定一个问题，并设置"num_generations=1"。

我们会先进行搜索，获取文章的相关上下文。简单起见我们选择只使用一个结果作为 prompt。例如，我们可能会使用这样的提示：“摘自吴恩达老师的文章《如何规划 AI 领域的职业生涯》。”通常，提供给模型的上下文越多，模型完成任务的效果就越好。

在此步骤中，我们将注入文章的段落作为上下文，即。

然后，我们向模型提出问题，要求它从提供的文本中提取答案。如果答案不在文本中，模型应告知我们答案不可得。

接下来，我们定义了要发送给模型的预测提示。使用"co.generate"函数，我们传入 prompt，并设置 max_tokens=70。

在某些情况下，我们可能会选择使用较长的模型。例如，我们可能会选择使用名为"command nightly"的模型。这是 Cohere 公司的生成模型，更新速度最快。因此，如果你使用"command nightly"，你将使用平台上最新的模型。虽然这些模型可能是实验性的，但它们通常是最新的，也是最好的。

最后，我们返回模型的生成结果 "prediction.generations"

In [87]:
def ask_andrews_article(question, num_generations=1):
    """
    根据给定的问题，从吴恩达老师的文章中生成答案。

    参数:
    question (str): 用户的 query。
    num_generations (int): 生成答案的次数，默认为1。

    返回:
    prediction.generations (list): 生成的答案列表。

    """

    # 检索文章
    results = search_andrews_article_zh(question)

    # 获取最佳结果
    context = results[0]

    # 准备 prompt
    prompt = f"""
    Excerpt from the article titled "How to Build a Career in AI"
    by Andrew Ng:
    {context}
    Question: {question}

    Extract the answer of the question from the text provided.
    If the text doesn't contain the answer,
    reply that the answer is not available."""

    # 生成预测
    prediction = co.generate(
        prompt=prompt,
        max_tokens=70,
        model="command-nightly",
        temperature=0.5,
        num_generations=num_generations
    )

    return prediction.generations

In [95]:
def ask_andrews_article_zh(question, num_generations=1):
    """
    根据给定的问题，从吴恩达老师的文章中生成答案。

    参数:
    question (str): 用户的 query。
    num_generations (int): 生成答案的次数，默认为1。

    返回:
    prediction.generations (list): 生成的答案列表。

    """

    # 检索文章
    results = search_andrews_article_zh(question)

    # 获取最佳结果
    context = results[0] # 中文效果有点差，可以手动指定 context 为英文的中文内容

    # 准备 prompt
    prompt = f"""

    摘自吴恩达老师的文章《如何规划 AI 领域的职业生涯》：
    {context}
    问题：{question}

    从提供的文本中提取问题的答案。
    如果文本不包含答案，
    回答答案不可用。"""
    # 生成预测
    prediction = co.generate(
        prompt=prompt,
        max_tokens=70,
        model="command-nightly",
        temperature=0.5,
        num_generations=num_generations
    )

    return prediction.generations


现在，让我们来向模型提问。

In [83]:
results = ask_andrews_article(
    "Are side projects a good idea when trying to build a career in AI?",

)

print(results[0])

default model on embed will be deprecated in the future, please specify a model in the request.


Side projects can be a great stepping stone to something significant, especially when there is no conflict with your employer, it can stir the creative juices and strengthen bonds with collaborators. 

So, the answer is yes, side projects are a good idea when trying to build a career in AI as they offer an avenue to explore unique and potentially impactful applications and


In [107]:
results = ask_andrews_article_zh(
    "在你开始探索人工智能的学习之旅时，实战项目对你的学习成效有多大的促进作用？"

)

print(results[0])

答案是：在人工智能领域的学习中，加入现有项目可以帮助学习者激发灵感，并通过与领域专家交流，获得更多学习资源。


这不仅仅是一个搜索练习，更是一个结合搜索和语言模型的对话练习。

这就是一个快速展示如何使用它的例子。你可以试着向它提问，有些问题可能需要稍微调整提问方式。这里给出一些高级应用的示例。

许多创新者正在尝试新的应用，例如，他们会向 Lex Fridman 的播客提出问题，这正是我们刚刚介绍的流程。他们会对播客的全文进行语义搜索。这种方法也被应用于安德鲁·胡伯曼的播客。你甚至可以在 YouTube 视频和书籍的文字记录中看到这种应用。因此，人们正在利用大型语言模型构建这样的应用，通常是通过搜索和生成的方式实现的，你也可以通过重新排名来优化搜索组件。现在，你可以暂停一下，尝试一下自己运行代码，并更改你想向模型提出的问题，或者获取你感兴趣的另一个数据集。你并不总是需要复制代码，这只是一个快速的示例。

如果你计划进行大规模的项目，可以考虑使用如
LlamaIndex 和 LangChain 等工具，这些工具可以帮助你从 PDF 文件中提取文本。此外，"num_generations"参数也是一个重要的考虑因素。在开发过程中，如果你想要测试模型对同一提示的多种反应，而不是每次都调用 API，那么使用"num_generations"参数是一个有效的策略。我们可以将这个参数传递给"code.generate"函数，即"num_generations=num_generations"。在提出问题时，我们可以设置"num_generations=3"。这样，我们就不仅仅得到一个结果，而是得到三个不同的生成结果。这意味着，我们提供的问题将被语言模型处理，模型将为我们提供三种不同的答案，这些答案将同时生成。

In [ ]:
results = ask_andrews_article(
    "Are side projects a good idea when trying to build a career in AI?",
    num_generations=3
)

for gen in results:
    print(gen)
    print('--')

 Yes, side projects are a good idea when trying to build a career in AI. They can help you develop your skills and knowledge, and can also be a stepping stone to a more substantive project. However, it is important to ensure that your side project does not create a conflict with your employer.
--
 Yes. Side projects are a good idea when trying to build a career in AI. They can help you to develop new ideas and to strengthen bonds with collaborators. They can also be a stepping stone to something more significant. However, it is important to note that you should not create a conflict with your employer.
--
 Yes. A side hustle can stir the creative juices and strengthen bonds with collaborators.
--


In [99]:
results = ask_andrews_article_zh(
    "在你开始探索人工智能的学习之旅时，实战项目对你的学习成效有多大的促进作用？",
    num_generations=3
)

for gen in results:
    print(gen)
    print('--')

答案是：加入现有项目可以帮助你学习，激发灵感，并可以帮助你探索机器学习的各种应用。同时，开发副业可以帮助你激发创造力，加强与其他人工智能开发者的合作。
--
人工智能的学习之旅中，加入现有项目、关注应用领域、开发副业等实战项目，可以帮助学习者激发创造力，加强与其他人工智能开发者的合作，从而对学习成效起到一定的促进作用。
--
答案是：人工智能的学习成效，在开始探索学习之旅时，需要通过加入现有项目，保持阅读和与人交谈，关注应用领域，开发副业等方式，来提高学习成效。
--


恭喜您完成课程！

要开始使用 Cohere LLMs 进行构建，请通过[此处](https://dashboard.cohere.ai/welcome/register?utm_source=partner&utm_medium=website&utm_campaign=DeeplearningAI)注册获取您的 API 密钥.

了解更多关于 LLMs 的信息在 Cohere 的 [LLM.University](https://LLM.University).